# Marchenko-Pastur Law: Sample Covariance Matrices

**Author:** Divyansh Atri

---

## The Marchenko-Pastur Law

Consider a data matrix $X \in \mathbb{R}^{n \times p}$ with i.i.d. entries $X_{ij} \sim N(0, \sigma^2)$.

The **sample covariance matrix** is:
$$W = \frac{1}{n} X^T X$$

This is a **Wishart matrix**. As $n, p \to \infty$ with $p/n \to \gamma$, the eigenvalue density converges to:

$$\rho(\lambda) = \frac{1}{2\pi \gamma \sigma^2 \lambda} \sqrt{(\lambda_+ - \lambda)(\lambda - \lambda_-)}$$

where $\lambda_\pm = \sigma^2(1 \pm \sqrt{\gamma})^2$.

This appears everywhere in high-dimensional statistics!

## My Goals

1. Generate Wishart matrices with different aspect ratios $\gamma = p/n$
2. Verify the Marchenko-Pastur law empirically
3. Study what happens when $\gamma < 1$ vs $\gamma > 1$
4. Explore the edge behavior

In [ ]:
# Setup
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt

from matrix_generators import generate_wishart_matrix
from eigenvalue_tools import compute_eigenvalues, eigenvalue_statistics
from spectral_density import (
    empirical_density,
    marchenko_pastur,
    compare_densities
)

np.random.seed(456)

## Experiment 1: Basic Verification

Let me start with a simple case: $n = 2000$, $p = 1000$, so $\gamma = 0.5$.

In [ ]:
n, p = 2000, 1000
gamma = p / n

print(f"Generating Wishart matrix: n={n}, p={p}, γ={gamma}")
W = generate_wishart_matrix(n, p)

print(f"Matrix shape: {W.shape}")
print("Computing eigenvalues...")
eigs = compute_eigenvalues(W)

# Theoretical support
lambda_minus = (1 - np.sqrt(gamma))**2
lambda_plus = (1 + np.sqrt(gamma))**2

print(f"\nTheoretical support: [{lambda_minus:.4f}, {lambda_plus:.4f}]")
print(f"Empirical range: [{eigs.min():.4f}, {eigs.max():.4f}]")

stats = eigenvalue_statistics(eigs)
print(f"\nMean eigenvalue: {stats['mean']:.4f}")
print(f"Expected mean: {1.0:.4f}")  # For normalized Wishart

In [ ]:
# Plot empirical vs theoretical
fig, ax = plt.subplots(figsize=(11, 6))

# Histogram
ax.hist(eigs, bins=50, density=True, alpha=0.6, 
        color='mediumseagreen', edgecolor='black', label='Empirical')

# Theoretical Marchenko-Pastur
x = np.linspace(0, lambda_plus * 1.2, 500)
y = marchenko_pastur(x, gamma)
ax.plot(x, y, 'r-', linewidth=2.5, label='Marchenko-Pastur')

# Mark the support
ax.axvline(lambda_minus, color='blue', linestyle='--', linewidth=1.5, 
           alpha=0.7, label=f'λ₋ = {lambda_minus:.3f}')
ax.axvline(lambda_plus, color='blue', linestyle='--', linewidth=1.5, 
           alpha=0.7, label=f'λ₊ = {lambda_plus:.3f}')

ax.set_xlabel('Eigenvalue λ', fontsize=12)
ax.set_ylabel('Density ρ(λ)', fontsize=12)
ax.set_title(f'Marchenko-Pastur Law (n={n}, p={p}, γ={gamma})', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../experiments/marchenko_pastur_basic.png', dpi=150, bbox_inches='tight')
plt.show()

print("Beautiful match!")

## Experiment 2: Varying the Aspect Ratio γ

The shape of the Marchenko-Pastur distribution changes with $\gamma = p/n$.  
Let me try different aspect ratios!

In [ ]:
# Different aspect ratios to test
n = 2000
gammas = [0.2, 0.5, 0.8, 1.5]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, gamma in enumerate(gammas):
    p = int(n * gamma)
    
    print(f"\nγ = {gamma}: n={n}, p={p}")
    W = generate_wishart_matrix(n, p)
    eigs = compute_eigenvalues(W)
    
    # Support
    lambda_minus = (1 - np.sqrt(gamma))**2
    lambda_plus = (1 + np.sqrt(gamma))**2
    
    ax = axes[idx]
    
    # Histogram
    ax.hist(eigs, bins=40, density=True, alpha=0.6, 
            color='mediumseagreen', edgecolor='black')
    
    # Theory
    x = np.linspace(max(0, lambda_minus - 0.2), lambda_plus + 0.2, 500)
    y = marchenko_pastur(x, gamma)
    ax.plot(x, y, 'r-', linewidth=2.5)
    
    ax.axvline(lambda_minus, color='blue', linestyle='--', linewidth=1.5, alpha=0.6)
    ax.axvline(lambda_plus, color='blue', linestyle='--', linewidth=1.5, alpha=0.6)
    
    ax.set_title(f'γ = {gamma} (p/n = {p}/{n})', fontsize=12, fontweight='bold')
    ax.set_xlabel('λ', fontsize=11)
    ax.set_ylabel('ρ(λ)', fontsize=11)
    ax.grid(alpha=0.3)

plt.suptitle('Marchenko-Pastur Law for Different Aspect Ratios', 
             fontsize=15, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('../experiments/marchenko_pastur_varying_gamma.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nAs γ changes, the distribution shape changes!")
print("Small γ: narrow support, concentrated near 1")
print("Large γ: wider support, more spread out")

## Experiment 3: The Special Case γ = 1

When $\gamma = 1$ (square data matrix, $n = p$), the Marchenko-Pastur distribution becomes:

$$\rho(\lambda) = \frac{1}{2\pi \lambda} \sqrt{(4 - \lambda) \lambda}, \quad 0 \leq \lambda \leq 4$$

This is a particularly nice case to study!

In [ ]:
n = p = 2000
gamma = 1.0

print(f"Square case: n = p = {n}, γ = {gamma}")
W = generate_wishart_matrix(n, p)
eigs = compute_eigenvalues(W)

fig, ax = plt.subplots(figsize=(11, 6))

ax.hist(eigs, bins=60, density=True, alpha=0.6, 
        color='darkorange', edgecolor='black', label='Empirical')

x = np.linspace(0, 4.5, 500)
y = marchenko_pastur(x, gamma)
ax.plot(x, y, 'r-', linewidth=2.5, label='Marchenko-Pastur (γ=1)')

ax.axvline(0, color='blue', linestyle='--', linewidth=1.5, alpha=0.6, label='λ₋ = 0')
ax.axvline(4, color='blue', linestyle='--', linewidth=1.5, alpha=0.6, label='λ₊ = 4')

ax.set_xlabel('Eigenvalue λ', fontsize=12)
ax.set_ylabel('Density ρ(λ)', fontsize=12)
ax.set_title(f'Marchenko-Pastur Law: Square Case (γ = 1)', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../experiments/marchenko_pastur_gamma_1.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nNotice the spike near λ = 0!")
print("This is characteristic of the Marchenko-Pastur distribution.")

## Experiment 4: Largest Eigenvalue Behavior

Similar to the Wigner case, the largest eigenvalue has interesting fluctuations.  
Let me study this empirically.

In [ ]:
# Parameters
n, p = 1000, 500
gamma = p / n
num_trials = 300

lambda_plus_theory = (1 + np.sqrt(gamma))**2

print(f"Running {num_trials} trials with n={n}, p={p}, γ={gamma}")
print(f"Theoretical edge: λ₊ = {lambda_plus_theory:.6f}")

max_eigenvalues = []

for i in range(num_trials):
    if (i + 1) % 50 == 0:
        print(f"  Trial {i + 1}/{num_trials}")
    
    W = generate_wishart_matrix(n, p)
    eigs = compute_eigenvalues(W)
    max_eigenvalues.append(eigs[-1])

max_eigenvalues = np.array(max_eigenvalues)

In [ ]:
# Plot distribution
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(max_eigenvalues, bins=25, density=True, alpha=0.6, 
        color='mediumvioletred', edgecolor='black')

ax.axvline(lambda_plus_theory, color='red', linestyle='--', linewidth=2, 
           label=f'Theoretical edge λ₊ = {lambda_plus_theory:.4f}')

mean_max = np.mean(max_eigenvalues)
ax.axvline(mean_max, color='blue', linestyle='-', linewidth=2, 
           label=f'Mean = {mean_max:.4f}')

ax.set_xlabel('Largest Eigenvalue λ_max', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title(f'Largest Eigenvalue Distribution (γ={gamma}, {num_trials} trials)', 
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../experiments/marchenko_pastur_edge.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nLargest eigenvalue statistics:")
print(f"  Mean: {mean_max:.6f}")
print(f"  Std:  {np.std(max_eigenvalues):.6f}")
print(f"  Theory: {lambda_plus_theory:.6f}")
print(f"  Deviation: {mean_max - lambda_plus_theory:.6f}")

## Experiment 5: Connection to PCA

In Principal Component Analysis (PCA), eigenvalues of the sample covariance tell us about the variance explained by each component.

The Marchenko-Pastur law tells us that **random noise** produces eigenvalues in $[\lambda_-, \lambda_+]$.  
Eigenvalues outside this range indicate **signal**!

Let me simulate a simple case with signal + noise.

In [ ]:
# Generate data: signal (rank 5) + noise
n, p = 2000, 1000
gamma = p / n
rank = 5
signal_strength = 10.0

print(f"Generating data: n={n}, p={p}")
print(f"Signal: rank {rank} with strength {signal_strength}")

# Signal part: low rank
U = np.random.randn(n, rank)
V = np.random.randn(rank, p)
Signal = (U @ V) * signal_strength / np.sqrt(n)

# Noise part
Noise = np.random.randn(n, p)

# Combined data
X = Signal + Noise

# Sample covariance
W = (X.T @ X) / n

print("Computing eigenvalues...")
eigs = compute_eigenvalues(W)

# Marchenko-Pastur edge
lambda_plus = (1 + np.sqrt(gamma))**2

print(f"\nMarchenko-Pastur edge: {lambda_plus:.4f}")
print(f"Largest eigenvalue: {eigs[-1]:.4f}")
print(f"Top 10 eigenvalues: {eigs[-10:]}")

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 6))

ax.hist(eigs, bins=60, density=True, alpha=0.6, 
        color='teal', edgecolor='black', label='Empirical (signal + noise)')

# Marchenko-Pastur for pure noise
x = np.linspace(0, max(eigs.max(), lambda_plus) + 1, 500)
y = marchenko_pastur(x, gamma)
ax.plot(x, y, 'r-', linewidth=2.5, label='Marchenko-Pastur (pure noise)')

# Mark the edge
ax.axvline(lambda_plus, color='orange', linestyle='--', linewidth=2, 
           label=f'Noise edge λ₊ = {lambda_plus:.3f}')

ax.set_xlabel('Eigenvalue λ', fontsize=12)
ax.set_ylabel('Density ρ(λ)', fontsize=12)
ax.set_title('Signal Detection: Eigenvalues Beyond Marchenko-Pastur Edge', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../experiments/marchenko_pastur_signal_detection.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nEigenvalues above the noise edge indicate signal!")
print("This is the basis for modern PCA-based signal detection methods.")

## Summary

In this notebook, I've explored the **Marchenko-Pastur law**:

1. ✅ Verified the distribution for sample covariance matrices
2. ✅ Studied how the shape changes with aspect ratio $\gamma$
3. ✅ Examined the special case $\gamma = 1$ (square matrices)
4. ✅ Investigated largest eigenvalue fluctuations
5. ✅ Demonstrated application to signal detection in noisy data

The Marchenko-Pastur law is crucial for:
- High-dimensional statistics
- PCA and covariance estimation
- Signal detection
- Random matrix theory applications

**Next:** Notebook 04 will study finite-size effects in detail.